In [1]:
import argparse
from intern.remote.boss import BossRemote
from intern.resource.boss.resource import *
from intern.utils.parallel import block_compute
import configparser
import requests
import numpy as np
from numpy import genfromtxt
import shutil
import blosc
from IPython.core.debugger import set_trace
import sys
import os
import itertools
from functools import partial
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import cpu_count
import csv
import datetime

In [3]:
def getData(di):
    data = di['rem'].get_cutout(di['ch_rsc'], di['res'], di['xrng'],
          di['yrng'] ,di['zrng'])
    out = data[di['mask']]
    return(out)


def getCentroid(box):
    m = np.asarray(box == True) 
    avg = np.int(np.round(np.mean(m,1)))
    return(avg)



In [2]:
COLL_NAME      = 'collman'
EXP_NAME       = 'collman15v2'
COORD_FRAME    = 'collman_collman15v2' 
OUTPUT         = 'output.txt'
CONFIG_FILE    = 'config.ini'

rem = BossRemote(CONFIG_FILE)

#CHAN_NAMES = rem.list_channels(COLL_NAME, EXP_NAME) 
CHAN_NAMES = ['DAPI1st', 'DAPI2nd', 'DAPI3rd', 'GABA488', 'GAD647',
              'gephyrin594', 'GS594', 'MBP488', 'NR1594', 'PSD95_488',
              'Synapsin647', 'VGluT1_647']


In [3]:
#def main(COLL_NAME, EXP_NAME, COORD_FRAME,
#         CHAN_NAMES=None, num_threads = 4, CONFIG_FILE= 'config.ini'):

bf = 5

config = configparser.ConfigParser()
config.read(CONFIG_FILE)
TOKEN = config['Default']['token']
boss_url = ''.join( ( config['Default']['protocol'],'://',config['Default']['host'],'/v1/' ) )
#print(boss_url)
#'https://api.boss.neurodata.io/v1/'

#intern
rem = BossRemote(CONFIG_FILE)

cf = CoordinateFrameResource(str(COLL_NAME + '_' + EXP_NAME))
cfr = rem.get_project(cf)
anno_res = ChannelResource('annotation', COLL_NAME, EXP_NAME, 'annotation', datatype='uint64')

ex = {'x': cfr.x_stop, 'y': cfr.y_stop, 'z': cfr.z_stop}

blocks = block_compute(0,ex['x'],0,ex['y'],0,ex['z'], origin = (0,0,0), block_size = (512, 512, 16))

rid = []
for b in blocks:
    rid = rid + rem.get_ids_in_region(anno_res, 0, b[0], b[1], b[2], [0,1])

u = np.unique(np.asarray(rid))

bb = [rem.get_bounding_box(anno_res, 0,ui, 'tight') for ui in u]

for i in range(len(bb)):
    bb[i]["id"] = u[i]

#    npbb = np.asarray(bb)
#    bbMax = np.maximum(bb)
#    print("Bounding Box max " + bbMax)
#
#    A = [rem.get_cutout(anno_res, 0, bb[i]["x_range"], bb[i]["y_range"],
#                        bb[i]["z_range"], id_list = [bb[i]['id']]) != 0 for i
#                        in range(len(bb))]
#
##    return(bb)

In [4]:
len(bb)
#bb = main(COLL_NAME, EXP_NAME, COORD_FRAME, 
#                       CHAN_NAMES=CHAN_NAMES, 
#                       num_threads = 4, CONFIG_FILE= CONFIG_FILE)
#

236

In [16]:
A = [np.int32(rem.get_cutout(anno_res, 0, bb[i]["x_range"], bb[i]["y_range"],
    bb[i]["z_range"], id_list = [bb[i]['id']])) != 0 for i in range(len(bb))]

In [19]:
A[0]

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ..., 
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ..., 
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ..., 
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, 

In [8]:
bbx = np.asarray([bb[i]['x_range'][1] - bb[i]['x_range'][0] for i in range(len(bb))])
bby = np.asarray([bb[i]['y_range'][1] - bb[i]['y_range'][0] for i in range(len(bb))])
bbz = np.asarray([bb[i]['z_range'][1] - bb[i]['z_range'][0] for i in range(len(bb))])

In [9]:
[np.max(bbx), np.max(bby), np.max(bbz)]

[345, 366, 14]

In [10]:
np.where(bbz == 14)

(array([82]),)

In [ ]:
ChanList = []

## For getting masked bounding box around centroid of annotation
for ch in CHAN_NAMES:
    di = [{
          'rem': rem,
          'ch_rsc':
            ChannelResource(ch,COLL_NAME,EXP_NAME,'image',datatype='uint8'),
          'ch'  : ch,
          'res' : 0,
          'xrng': [Bcon[i][2] - bf[2], Bcon[i][2] + bf[2] + 1], 
          'yrng': [Bcon[i][1] - bf[1], Bcon[i][1] + bf[1] + 1], 
          'zrng': [Bcon[i][0] - bf[0], Bcon[i][0] + bf[0] + 1], 
          'mask': AA[i]
         } for i in range(len(Bcon))]
    with ThreadPool(num_threads) as tp:
        out = tp.map(toolbox.getMaskDataT, di)
        print(ch) ## STATUS
        sys.stdout.flush() ## STATUS
    ChanList.append(np.asarray(out))

cubes = np.asarray(ChanList)
loc = np.asarray(Bcon)
nonzeros = [np.sum(ai) for ai in AA]
ids = [conID for i in range(len(conID))]

